# анализ данных фитнес-трекера

этот ноутбук подключается к postgresql и анализирует данные с фитнес-устройств

In [ ]:
# установка необходимых библиотек
!pip install psycopg2-binary sqlalchemy pandas matplotlib seaborn -q

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# настройка отображения графиков
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

## подключение к базе данных

In [ ]:
# параметры подключения к postgresql
# при запуске в docker используем имя сервиса postgres
DB_HOST = 'postgres'
DB_PORT = 5432
DB_NAME = 'fitness_db'
DB_USER = 'fitness'
DB_PASSWORD = 'fitness123'

# создаем подключение
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
print('подключение к базе данных установлено')

## загрузка данных

In [ ]:
# загружаем все данные из таблицы fitness_events
query = """
SELECT id, timestamp, user_id, steps, heart_rate, calories, activity_type
FROM fitness_events
ORDER BY timestamp DESC
"""

df = pd.read_sql(query, engine)
print(f'загружено записей: {len(df)}')
df.head(10)

## базовая статистика

In [ ]:
# описательная статистика числовых полей
df[['steps', 'heart_rate', 'calories']].describe()

In [ ]:
# распределение по типам активности
activity_counts = df['activity_type'].value_counts()
print('количество записей по типам активности:')
print(activity_counts)

## визуализация данных

In [ ]:
# круговая диаграмма типов активности
fig, ax = plt.subplots(figsize=(8, 6))
activity_counts.plot(kind='pie', autopct='%1.1f%%', ax=ax)
ax.set_ylabel('')
ax.set_title('распределение типов активности')
plt.tight_layout()
plt.show()

In [ ]:
# средние показатели по типам активности
activity_stats = df.groupby('activity_type').agg({
    'steps': 'mean',
    'heart_rate': 'mean',
    'calories': 'mean'
}).round(2)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

activity_stats['steps'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('средние шаги по активности')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)

activity_stats['heart_rate'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('средний пульс по активности')
axes[1].set_xlabel('')
axes[1].tick_params(axis='x', rotation=45)

activity_stats['calories'].plot(kind='bar', ax=axes[2], color='forestgreen')
axes[2].set_title('средние калории по активности')
axes[2].set_xlabel('')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# статистика по пользователям
user_stats = df.groupby('user_id').agg({
    'steps': 'sum',
    'calories': 'sum',
    'heart_rate': 'mean',
    'id': 'count'
}).rename(columns={'id': 'records_count'})

print('статистика по пользователям:')
user_stats

In [ ]:
# суммарные показатели по пользователям
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

user_stats['steps'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('суммарные шаги по пользователям')
axes[0].set_xlabel('пользователь')
axes[0].set_ylabel('шаги')

user_stats['calories'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('суммарные калории по пользователям')
axes[1].set_xlabel('пользователь')
axes[1].set_ylabel('калории')

plt.tight_layout()
plt.show()

## корреляционный анализ

In [ ]:
# матрица корреляций между числовыми показателями
correlation_matrix = df[['steps', 'heart_rate', 'calories']].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=ax)
ax.set_title('корреляция между показателями')
plt.tight_layout()
plt.show()

In [ ]:
# scatter plot пульса и калорий
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(data=df, x='heart_rate', y='calories', hue='activity_type', alpha=0.6, ax=ax)
ax.set_title('зависимость калорий от пульса по типам активности')
ax.set_xlabel('пульс (уд/мин)')
ax.set_ylabel('калории')
plt.legend(title='активность', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## анализ временных рядов

In [ ]:
# пульс во времени (последние 100 записей)
recent_data = df.head(100).sort_values('timestamp')

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(recent_data['timestamp'], recent_data['heart_rate'], marker='o', markersize=3, alpha=0.7)
ax.set_title('динамика пульса (последние 100 записей)')
ax.set_xlabel('время')
ax.set_ylabel('пульс (уд/мин)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## выводы

на основе анализа данных фитнес-трекера можно сделать следующие выводы:

1. **типы активности** - распределение активностей соответствует ожидаемым весам генератора
2. **пульс** - наблюдается четкая корреляция между типом активности и пульсом
3. **калории** - количество сожженных калорий напрямую зависит от интенсивности активности
4. **шаги** - максимальное количество шагов при беге, минимальное при сне и велосипеде